In [1]:
import os
os.chdir("..")

In [2]:
import json
import numpy as np
import statistics
import math
import pandas as pd

## Load climate change contrarianism data

In [4]:
with open('./data/bisection_records/CCC_BART.json', 'r') as file:
    clim_record = json.load(file)

In [5]:
df_clim = pd.read_pickle('./data/climate_change/training.pkl')

In [7]:
df_clim.columns

Index(['text', 'claim', '1_1_annot', '1_2_annot', '1_3_annot', '1_4_annot',
       '1_6_annot', '1_7_annot', 'FSL_BART', 'multi_annot', 'sbert_cosine',
       'FSL_DistilBART', 'FSL_DeBERTa', 'FSL_BART_neg', 'FSL_BART_extras'],
      dtype='object')

In [8]:
df_clim["labels_list"] = df_clim["claim"].apply(lambda x: [t for t in clim_record if x == t[:3]])

In [9]:
with open('./data/climate_change/claims.json') as file:
    all_traits = json.load(file)

In [10]:
traits_clim = {k: all_traits[k] for k in clim_record}

## Load SemEval data

In [11]:
with open('./data/bisection_records/TS_BART_topic.json', 'r') as file:
    semeval_topic_record = json.load(file)
    
with open('./data/bisection_records/TS_BART_stance.json', 'r') as file:
    semeval_stance_record = json.load(file)

In [12]:
df_semeval = pd.read_pickle('./data/topic_stance/training.pkl')

In [13]:
df_semeval["FSL_BART"] = df_semeval.apply(lambda x: {**x["FSL_BART_topic"], **x["FSL_BART_stance"]}, axis=1)

In [14]:
stances_descr = {
    "1A": "Atheism - AGAINST",
    "1F": "Atheism - FAVOR",
    "2A": "Climate Change is a Real Concern - AGAINST",
    "2F": "Climate Change is a Real Concern - FAVOR",
    "3A": "Feminist Movement - AGAINST",
    "3F": "Feminist Movement - FAVOR",
    "4A": "Hillary Clinton - AGAINST",
    "4F": "Hillary Clinton - FAVOR",
    "5A": "Legalization of Abortion - AGAINST",
    "5F": "Legalization of Abortion - FAVOR",
}

topics_descr = {
    "1": "Atheism",
    "2": "Climate Change is a Real Concern",
    "3": "Feminist Movement",
    "4": "Hillary Clinton",
    "5": "Legalization of Abortion",
}

In [16]:
df_semeval["labels_list"] = df_semeval.apply(lambda x: [t for t in semeval_topic_record if x["Target"] == topics_descr[t[0]]] + [t_2 for t_2 in semeval_stance_record if x["Stance"] != "NONE" and x["Target"] + " - " + x["Stance"] == stances_descr[t_2[:2]]], axis=1)

In [17]:
with open('./data/topic_stance/claims_topic.json') as file:
    traits_topics = json.load(file)

with open('./data/topic_stance/claims_stance.json') as file:
    traits_stance = json.load(file)

## Load depression data

In [18]:
with open('./data/bisection_records/D_BART.json', 'r') as file:
    dep_record = json.load(file)

In [19]:
classes = sorted(list(set([k.split("_")[0] for k in dep_record])))
class2idx = {c:i for i,c in enumerate(classes)}

In [21]:
df_dep = pd.read_pickle('./data/depression/training.pkl')

In [23]:
df_dep["labels_list"] = df_dep["multi_labels"].apply(lambda x: [t for t in dep_record if x[class2idx[t.split("_")[0]]]])

In [25]:
with open('./data/depression/claims.json') as file:
    traits_dep = json.load(file)

## Generate report

In [29]:
start = 0
stop = 1
step = 0.01
x = np.arange(start, stop, step)

def find_CI(distrib, low_mass=0.025, high_mass=0.975):
    exp_f = np.exp(distrib)
    alpha_low = exp_f.sum() * low_mass
    alpha_high = exp_f.sum() * high_mass

    try:
        l_bound_low = x[exp_f.cumsum() <= alpha_low][-1]
        l_low_weight = exp_f.cumsum()[int(l_bound_low * 100)]
    except IndexError:
        l_bound_low = start
        l_low_weight = 0.0
        
    try:
        l_bound_high = x[::-1][exp_f[::-1].cumsum() <= alpha_high][-1]
        l_high_weight = exp_f.cumsum()[int(l_bound_high * 100)]
    except IndexError:
        l_bound_high = stop
        l_high_weight = 1.0
        
    l_bound_avg = (l_bound_low + l_bound_high) / 2
    
    if (l_bound_avg * 1000) % 10 != 0:
        round_down = math.floor(int(l_bound_avg * 100))
        round_up = math.ceil(int(l_bound_avg * 100))
        l_weight = (exp_f.cumsum()[round_down] + exp_f.cumsum()[round_up]) / 2
    else:
        l_weight = exp_f.cumsum()[int(l_bound_avg * 100)]  
        
    try:
        r_bound_low = x[exp_f.cumsum() <= alpha_high][-1]
        r_low_weight = exp_f.cumsum()[int(r_bound_low * 100)]
    except IndexError:
        r_bound_low = start
        r_low_weight = 0.0
        
    try:
        r_bound_high = x[::-1][exp_f[::-1].cumsum() <= alpha_low][-1]
        r_high_weight = exp_f.cumsum()[int(r_bound_high * 100)]
    except IndexError:
        r_bound_high = stop
        r_high_weight = 1.0
        
    r_bound_avg = (r_bound_low + r_bound_high) / 2
    
    if (r_bound_avg * 1000) % 10 != 0:
        round_down = math.floor(int(r_bound_avg * 100))
        round_up = math.ceil(int(r_bound_avg * 100))
        r_weight = (exp_f.cumsum()[round_down] + exp_f.cumsum()[round_up]) / 2
    else:
        r_weight = exp_f.cumsum()[int(r_bound_avg * 100)]  

    return l_bound_avg, r_bound_avg

In [36]:
# Average number of annotations
def print_report(record, df_train, traits):
    nb_annots = list()
    ci_width = list()
    sample_size = list()
    thresholds_all = list()

    conv_claims = list()
    conv_nb_annots = list()
    conv_ci_width = list()
    conv_sample_size = list()
    conv_thresholds = list()

    not_conv_claims = list()
    not_conv_nb_annots = list()
    not_conv_ci_width = list()
    not_conv_sample_size = list()
    not_conv_thresholds = list()

    for t in record:
        nb = len(record[t]["annot"])
        nb_annots.append(nb)
        l_bound_1, r_bound_1 = find_CI(record[t]['distributions'][-1], low_mass=0.05, high_mass=0.95)
        w = r_bound_1 - l_bound_1
        ci_width.append(w)
        df_sub = df_train[df_train["labels_list"].apply(lambda x: t in x)]
        sample_size.append(len(df_sub))
        med_l, med_h = find_CI(record[t]['distributions'][-1], low_mass=0.5, high_mass=0.5)
        median = (med_l + med_h) / 2
        thresholds_all.append(median)

        df_train["X"] = df_train["FSL_BART"].apply(lambda x: x[traits[t]])
        df_train["Y"] = df_train["labels_list"].apply(lambda x: 1 if t in x else 0)
        
        if w > 0.20:
            not_conv_claims.append(t)
            not_conv_nb_annots.append(nb)
            not_conv_ci_width.append(w)
            not_conv_sample_size.append(len(df_sub))
            not_conv_thresholds.append(median)
        else:
            conv_claims.append(t)
            conv_nb_annots.append(nb)
            conv_ci_width.append(w)
            conv_sample_size.append(len(df_sub))
            conv_thresholds.append(median)

    nb_claims = len(record)
    conv_nb_claims = len(conv_claims)
    not_conv_nb_claims = len(not_conv_claims)

    print("***** ALL *****")
    print()
    print("Nb claims:\t\t", nb_claims)
    print("Average nb annots:\t %.2f" % (sum(nb_annots) / nb_claims))
    print("Average CI width:\t %.2f" % (sum(ci_width) / nb_claims))
    print("Average sample size:\t %.2f" % (sum(sample_size) / nb_claims))
    print("Average threshold:\t %.2f" % (sum(thresholds_all) / nb_claims))
    print("Std average threshold:\t %.2f" % (np.std(thresholds_all)))
    print("Max threshold:\t\t %.2f" % (max(thresholds_all)))
    print("Min threshold:\t\t %.2f" % (min(thresholds_all)))
    print()

    print("***** COMPLETE *****")
    print()
    print("Nb claims:\t\t", conv_nb_claims)
    print("Average nb annots:\t %.2f" % (sum(conv_nb_annots) / conv_nb_claims))
    print("Average CI width:\t %.2f" % (sum(conv_ci_width) / conv_nb_claims))
    print("Average sample size:\t %.2f" % (sum(conv_sample_size) / conv_nb_claims))
    print("Average threshold:\t %.2f" % (sum(conv_thresholds) / nb_claims))
    print("Std average threshold:\t %.2f" % (np.std(conv_thresholds)))
    print("Max threshold:\t\t %.2f" % (max(conv_thresholds)))
    print("Min threshold:\t\t %.2f" % (min(conv_thresholds)))
    print()

    print("***** EARLY STOP *****")
    print()
    print("Nb claims:\t\t", not_conv_nb_claims)
    print("Average nb annots:\t %.2f" % (sum(not_conv_nb_annots) / not_conv_nb_claims))
    print("Average CI width:\t %.2f" % (sum(not_conv_ci_width) / not_conv_nb_claims))
    print("Average sample size:\t %.2f" % (sum(not_conv_sample_size) / not_conv_nb_claims))
    print("Average threshold:\t %.2f" % (sum(not_conv_thresholds) / nb_claims))
    print("Std average threshold:\t %.2f" % (np.std(not_conv_thresholds)))
    print("Max threshold:\t\t %.2f" % (max(not_conv_thresholds)))
    print("Min threshold:\t\t %.2f" % (min(not_conv_thresholds)))
    print()

## Climate Change Contrarianism detection task

In [37]:
print_report(clim_record, df_clim, traits_clim)

***** ALL *****

Nb claims:		 30
Average nb annots:	 19.63
Average CI width:	 0.38
Average sample size:	 298.33
Average threshold:	 0.76
Std average threshold:	 0.17
Max threshold:		 0.99
Min threshold:		 0.34

***** COMPLETE *****

Nb claims:		 9
Average nb annots:	 19.56
Average CI width:	 0.12
Average sample size:	 310.78
Average threshold:	 0.28
Std average threshold:	 0.04
Max threshold:		 0.99
Min threshold:		 0.86

***** EARLY STOP *****

Nb claims:		 21
Average nb annots:	 19.67
Average CI width:	 0.49
Average sample size:	 293.00
Average threshold:	 0.47
Std average threshold:	 0.14
Max threshold:		 0.90
Min threshold:		 0.34



## Topic/Stance classification task

In [38]:
print_report({ **semeval_topic_record, **semeval_stance_record}, df_semeval, {**traits_topics, **traits_stance})

***** ALL *****

Nb claims:		 85
Average nb annots:	 21.11
Average CI width:	 0.41
Average sample size:	 278.80
Average threshold:	 0.53
Std average threshold:	 0.27
Max threshold:		 0.99
Min threshold:		 0.05

***** COMPLETE *****

Nb claims:		 15
Average nb annots:	 29.60
Average CI width:	 0.11
Average sample size:	 264.73
Average threshold:	 0.10
Std average threshold:	 0.42
Max threshold:		 0.99
Min threshold:		 0.05

***** EARLY STOP *****

Nb claims:		 70
Average nb annots:	 19.29
Average CI width:	 0.47
Average sample size:	 281.81
Average threshold:	 0.43
Std average threshold:	 0.23
Max threshold:		 0.95
Min threshold:		 0.10



## Depressive symptoms detection task

In [40]:
print_report(dep_record, df_dep, traits_dep)

***** ALL *****

Nb claims:		 64
Average nb annots:	 11.16
Average CI width:	 0.23
Average sample size:	 31.44
Average threshold:	 0.93
Std average threshold:	 0.08
Max threshold:		 0.99
Min threshold:		 0.61

***** COMPLETE *****

Nb claims:		 42
Average nb annots:	 13.21
Average CI width:	 0.08
Average sample size:	 32.57
Average threshold:	 0.64
Std average threshold:	 0.02
Max threshold:		 0.99
Min threshold:		 0.91

***** EARLY STOP *****

Nb claims:		 22
Average nb annots:	 7.23
Average CI width:	 0.52
Average sample size:	 29.27
Average threshold:	 0.29
Std average threshold:	 0.09
Max threshold:		 0.95
Min threshold:		 0.61

